# Neural Networks

## Macros

### Imports

In [1]:
import tensorflow as tf
from os.path import join
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import plotly.express as px
import pandas as pd

2025-08-21 14:25:41.480363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755797141.494470   30715 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755797141.498511   30715 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-21 14:25:41.512066: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Defines

In [2]:
DATA_PATH=join("..", "..", "..", "resources", "database", "Vegetable_market.csv")

data = pd.read_csv(DATA_PATH)

## GPU Test

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.config.list_logical_devices()

I0000 00:00:1755797144.244508   30715 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2609 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## Train Model

In [5]:
cat_cols = ["Vegetable", "Season", "Month", "Deasaster Happen in last 3month", "Vegetable condition"]

for col in cat_cols:
    encoder = LabelEncoder()
    data[col] = encoder.fit_transform(data[col])

data

,Vegetable,Season,Month,Temp,Deasaster Happen in last 3month,Vegetable condition,Price per kg
0,13,4,4,15,0,1,20
1,16,4,4,15,0,1,50
2,11,4,4,15,0,1,70
3,14,4,4,15,0,1,25
4,6,4,4,15,0,1,20
...,...,...,...,...,...,...,...
116,2,4,4,15,1,1,33
117,8,4,4,15,0,1,88
118,13,3,1,32,0,1,24
119,11,3,1,33,0,1,33


In [6]:
x = data.iloc[:, :6]
y = data["Price per kg"]

In [7]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(x)
y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=42)

In [9]:
def model_builder(hp) -> tf.keras.Sequential:
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation="relu"))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="linear"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[
            tf.keras.metrics.MeanSquaredError()
        ]
    )
    
    return model

In [10]:
tuner = kt.Hyperband(
    model_builder,
    objective="mean_squared_error",
    max_epochs=30,
    factor=3,
    directory='Tuner',
    project_name='Tuning'
)

Reloading Tuner from Tuner/Tuning/tuner0.json


In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [12]:
tuner.search(
    x_train,
    y_train,
    epochs=30,
    validation_data = (x_test, y_test),
    callbacks=[stop_early]
)

In [13]:
hyperparms = tuner.get_best_hyperparameters(num_trials=1)[0]

In [14]:
model = tuner.hypermodel.build(hyperparms)

In [15]:
history_training = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50


I0000 00:00:1755797145.546509   31154 service.cc:148] XLA service 0x7f502c005750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755797145.546528   31154 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-08-21 14:25:45.568214: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755797145.657900   31154 cuda_dnn.cc:529] Loaded cuDNN version 90101


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0937 - mean_squared_error: 1.0937

I0000 00:00:1755797146.656995   31154 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 629ms/step - loss: 1.2048 - mean_squared_error: 1.2048 - val_loss: 1.7220 - val_mean_squared_error: 1.7220
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.7175 - mean_squared_error: 1.7175 - val_loss: 0.7764 - val_mean_squared_error: 0.7764
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7051 - mean_squared_error: 0.7051 - val_loss: 0.6519 - val_mean_squared_error: 0.6519
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.6747 - mean_squared_error: 0.6747 - val_loss: 0.7226 - val_mean_squared_error: 0.7226
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5800 - mean_squared_error: 0.5800 - val_loss: 0.8398 - val_mean_squared_error: 0.8398
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6842 - mean_squared_error: 0.6842 - val_loss: 0.7812 - val_mean_squared_error: 0.7812
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5612 - mean_squared_error: 0.5612 - val_loss: 0.6842 - val_mean_squared_error: 0.

In [16]:
mse_history = pd.DataFrame({
    "Treino": history_training.history["mean_squared_error"],
    "Teste": history_training.history["val_mean_squared_error"]})

fig = px.line(
    data_frame=mse_history,
    markers=True
)

fig.update_layout(
    xaxis_title="Epoch",
    yaxis_title="MSE",
    title="MSE x Epoch"
)

In [17]:
loss_history = pd.DataFrame({
    "Treino": history_training.history["loss"],
    "Teste": history_training.history["val_loss"]})

fig = px.line(
    data_frame=mse_history,
    markers=True
)

fig.update_layout(
    xaxis_title="Epoch",
    yaxis_title="Loss",
    title="Loss Function x Epoch"
)

In [18]:
evaluate = model.evaluate(x_test, y_test)
evaluate_df = pd.DataFrame({"Metric": ["Loss", "MSE"], "Values": evaluate})

px.bar(
    data_frame=evaluate_df,
    x="Metric",
    y="Values",
    color="Metric",
    title="Evaluate Metrics"
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - loss: 0.2619 - mean_squared_error: 0.2619


In [19]:
pred = model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


In [20]:
pred_df = pd.DataFrame({
    "Real": y_scaler.inverse_transform(y_test).reshape(-1, 1).ravel(),
    "Predict": y_scaler.inverse_transform(pred).reshape(-1, 1).ravel()
})

fig = px.line(pred_df,
        markers=True)

fig.update_layout(
    xaxis_title="Vegetable",
    yaxis_title="Price",
    title="Predict price X Real price"
)

fig

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 320)            │         2,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 320)            │       102,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           321 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 315,845 (1.20 MB)

 Trainable params: 105,281 (411.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 210,564 (822.52 KB)

In [23]:
model.save("NN_vegetable_price.h5")